In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import time

# ==========================================
# 1. DEFINICIÓN DEL MODELO M-M+TER
# ==========================================

class HumorBETO_MM(nn.Module):
    def __init__(self, freeze_bert=False):
        super(HumorBETO_MM, self).__init__()
        
        # Cargar BETO (Pre-entrenado en español)
        print("Cargando BETO...")
        self.bert = BertModel.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
        
        # Opcional: Congelar pesos de BERT para entrenar más rápido al principio
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
        # Dimensiones:
        # BETO output por token = 768
        # M-M Pooling concatena (Mean + Max) = 768 + 768 = 1536
        self.input_dim = 768 * 2 
        
        # Clasificador MLP final
        self.classifier = nn.Sequential(
            nn.Linear(self.input_dim, 512),
            nn.BatchNorm1d(512),
            nn.GELU(),         # GELU va mejor con BERT que ReLU
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(128, 2)  # Salida binaria
        )

    def forward(self, input_ids, attention_mask):
        # 1. Paso por BETO
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        # last_hidden_state tiene forma (Batch, Secuencia, 768)
        # Contiene el vector de CADA palabra
        hidden_state = outputs.last_hidden_state 
        
        # 2. IMPLEMENTACIÓN DE M-M POOLING (Manual para precisión)
        # Necesitamos enmascarar el padding para que no afecte al promedio ni al máximo
        
        # Expandir máscara para que coincida con dimensiones (Batch, Secuencia, 768)
        mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_state.size()).float()
        
        # A. Mean Pooling (Promedio ignorando ceros de padding)
        sum_embeddings = torch.sum(hidden_state * mask_expanded, 1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9) # Evitar división por cero
        mean_pooled = sum_embeddings / sum_mask
        
        # B. Max Pooling (Máximo ignorando ceros de padding)
        # Ponemos un valor muy negativo donde hay padding para que el max no sea 0
        hidden_state[mask_expanded == 0] = -1e9
        max_pooled = torch.max(hidden_state, 1)[0]
        
        # 3. Concatenación (La parte "M-M")
        concat_vector = torch.cat((mean_pooled, max_pooled), 1) # Dimensión 1536
        
        # 4. Clasificación
        logits = self.classifier(concat_vector)
        
        return logits

# ==========================================
# 2. PREPARACIÓN DE DATOS (Dataset Class)
# ==========================================

class HumorTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# ==========================================
# 3. CONFIGURACIÓN Y CARGA
# ==========================================

# A. Parámetros
MAX_LEN = 100        # Longitud máxima del tweet (ajustar según tus datos)
BATCH_SIZE = 16      # BERT consume mucha RAM, 16 o 32 suele ser el límite en Colab
EPOCHS = 4
LR = 2e-5            # Learning rate muy bajo (estándar para fine-tuning)

# B. Cargar Datos (IMPORTANTE: CAMBIA LA RUTA AL JSON CON TEXTO)
# Asumo que el JSON tiene una columna 'text' o 'tweet' con el texto original
print("Leyendo datos...")
df = pd.read_json("../Datasets/dataset_humor_train.json", lines=True) # <-- OJO AQUÍ

# Ajusta el nombre de la columna si es necesario (ej: 'tweet', 'text', 'contenido')
textos = df['text'].values 
etiquetas = df['klass'].values

# Split
X_train, X_val, y_train, y_val = train_test_split(textos, etiquetas, test_size=0.15, stratify=etiquetas, random_state=42)

# C. Tokenizer
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

# D. DataLoaders
train_dataset = HumorTextDataset(X_train, y_train, tokenizer, MAX_LEN)
val_dataset = HumorTextDataset(X_val, y_val, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# ==========================================
# 4. BUCLE DE ENTRENAMIENTO
# ==========================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

model = HumorBETO_MM(freeze_bert=False) # False = Fine-tuning completo (Mejores resultados)
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=LR, correct_bias=False)
total_steps = len(train_loader) * EPOCHS

# Scheduler con Warmup (ayuda a estabilizar BERT al inicio)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

# --- ENTRENAMIENTO ---
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    print("-" * 10)
    
    # Train
    model.train()
    losses = []
    correct_predictions = 0
    
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        targets = batch["labels"].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        loss = loss_fn(outputs, targets)
        losses.append(loss.item())
        
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # Evitar explosión de gradientes
        optimizer.step()
        scheduler.step()
        
    print(f"Train loss: {np.mean(losses)}")
    
    # Validation
    model.eval()
    val_losses = []
    predictions = []
    real_values = []
    
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            targets = batch["labels"].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs, targets)
            val_losses.append(loss.item())
            
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            real_values.extend(targets.cpu().tolist())
    
    val_f1 = f1_score(real_values, predictions, average='macro')
    print(f"Val Loss: {np.mean(val_losses)}")
    print(f"Val F1 Macro: {val_f1:.4f}")

    # Guardar el mejor modelo manualmente si supera cierto umbral
    if val_f1 > 0.84:
        torch.save(model.state_dict(), f'beto_mm_epoch_{epoch+1}_f1_{val_f1:.3f}.bin')

print("Entrenamiento finalizado.")